In [ ]:
import pandas as pd
import networkx as nx

In [ ]:
from model.post import Post, PostType

In [ ]:
prep_df = pd.read_json("data/preprocessed_data.jsonl", orient="records")

posts = prep_df.to_dict("records")

assert len(posts) == len(prep_df), "Unexpectedly dropped records"

In [ ]:
def unmarshal_post(record):
    return Post(
        post_id=record["index"],
        blog_url=record["blog_url"],
        post_type=PostType[record["type"].upper()],
        lang=record["lang"],
        is_reblog=(int(record["is_reblog"]) == 1),
        tags=record["tags"],
        root_tags=record["root_tags"]
    )

model_posts = list(map(unmarshal_post, posts))

In [ ]:
from collections import defaultdict

co_occurring_tags = defaultdict(float)
for post in toy_posts:
    for i in range(len(post.tags)):
        for j in range(len(post.tags)):
            if i == j:
                continue
            else:
                key = tuple(sorted([post.tags[i], post.tags[j]]))
                co_occurring_tags[key] += (1 / len(post.tags))

In [ ]:
post2post = nx.Graph()

for post in toy_posts:
    sorted_tags = sorted(post.tags)
    num_tags = len(sorted_tags)
    weight = 1 / num_tags
    for i in range(num_tags):
        for j in range(i+1, num_tags):
            
            a, b = sorted_tags[i], sorted_tags[j]
            if (a, b) in post2post.edges:
                new_weight = post2post.edges[a, b]['weight'] + weight
                post2post.add_edge(a, b, weight=new_weight)
            else:
                post2post.add_edge(a, b, weight=weight)

In [ ]:
G = post2post

In [ ]:
def prune(G, key, k = 25):
    return sorted(
        G[key].items(), 
        key=lambda x: x[1]["weight"], 
        reverse=True
    )[:k]

def random_sample(G, key, k = 25):
    pruned = prune(G, key, k)
    pop, weights = zip(*[
        (neighbor, features['weight'])
        for neighbor, features in pruned
    ])
    return random.choices(pop, weights, k=k)

random_sample(G, "star wars")

In [ ]:
pruned = sorted(list(G["star wars"].items()), key=lambda x: x[1], reverse=True)

pop, weights = zip(*[
    (neighbor, features['weight'])
    for neighbor, features in pruned
])

In [ ]:
random.choices(pop, weights, k=10)

In [ ]:
random.choice(posts)

In [ ]:
quota = 10
count = 0
for post in model_posts:
    if len(post.root_tags) == 0:
        if quota > 0:
            print(post)
            quota -= 1
        count += 1
count, count / len(model_posts)